In [1]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 37339 (delta 11), reused 11 (delta 3), pack-reused 37301
Receiving objects: 100% (37339/37339), 27.27 MiB | 24.24 MiB/s, done.
Resolving deltas: 100% (25827/25827), done.


In [2]:
cd transformers

/kaggle/working/transformers


In [3]:
!pip install .

Processing /kaggle/working/transformers
     |████████████████████████████████| 3.0 MB 4.8 MB/s 
  Created wheel for transformers: filename=transformers-3.0.2-py3-none-any.whl size=842108 sha256=c39921e91dd49341957ea19b28a9f4892d6a53bc88502ea34a9550ecbaeaefc7
  Stored in directory: /tmp/pip-ephem-wheel-cache-ota_3opg/wheels/be/1e/28/7186a3baa6fcb4e9201f390b70b4e6d75651e85d4e8a9ae413
Successfully built transformers
ERROR: allennlp 1.0.0 has requirement transformers<2.12,>=2.9, but you'll have transformers 3.0.2 which is incompatible.
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.7.0
    Uninstalling tokenizers-0.7.0:
      Successfully uninstalled tokenizers-0.7.0
  Attempting uninstall: transformers
    Found existing installation: transformers 2.11.0
    Uninstalling transformers-2.11.0:
      Successfully uninstalled transformers-2.11.0
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
import logging
import math
import os
from dataclasses import dataclass, field
from typing import Optional, List, Union
from torchtext.datasets import WikiText2
from transformers import GPT2Tokenizer
from transformers import Trainer
import torch
from transformers import GPT2LMHeadModel
import torch.optim as optim
from spacy.lang.en import English
from sklearn.model_selection import train_test_split
from torch.utils.data.dataset import Dataset

from transformers import (
PreTrainedModel, PreTrainedTokenizer, LineByLineTextDataset,
TrainingArguments, BatchEncoding, DataCollatorForLanguageModeling
)

from typing import Any, Callable, Dict, List, NewType, Tuple, Union
import torch
from torch.nn.utils.rnn import pad_sequence

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [5]:
class CustomDataset(Dataset):
    def __init__(self, examples:list):
        self.examples = examples
    
    def __len__(self):
        return(len(self.examples))
    
    def __getitem__(self, i):
        return(self.examples[i])
    

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", pad_token='<|endoftext|>', unk_token='<|endoftext|>')
model = GPT2LMHeadModel.from_pretrained("gpt2")
model = model.to(device)

In [8]:
model_max_input = tokenizer.max_model_input_sizes['gpt2']
print(model_max_input)

1024


In [9]:
FILE_NAME = "/kaggle/input/adventures-of-sherlock-holmes/adventure_of_sherlock_homles.txt"

In [10]:
data_set = LineByLineTextDataset(tokenizer, FILE_NAME, 45)

In [11]:
tr, te = train_test_split(data_set, test_size=0.1)

In [12]:
tr = CustomDataset(tr)

In [13]:
te = CustomDataset(te)

In [14]:
training_args = TrainingArguments(
    output_dir='/kaggle/working/results/',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='/kaggle/working/logs/',            # directory for storing logs
    evaluate_during_training=True,
    eval_steps=500,
    save_total_limit=1,
    
)

In [15]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [16]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    data_collator=data_collator,
    train_dataset=tr,         # training dataset
    eval_dataset=te,            # evaluation dataset
    prediction_loss_only=True,
)

In [17]:
trainer.train()

{'loss': 3.964815472126007, 'learning_rate': 5e-05, 'epoch': 0.922509225092251, 'step': 500}



{'eval_loss': 3.5270036204916533, 'epoch': 0.922509225092251, 'step': 500}


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{'loss': 3.4430290875434877, 'learning_rate': 2.7797513321492007e-05, 'epoch': 1.8450184501845017, 'step': 1000}



{'eval_loss': 3.449839623247991, 'epoch': 1.8450184501845017, 'step': 1000}



{'loss': 3.1874004073143007, 'learning_rate': 5.595026642984015e-06, 'epoch': 2.767527675276753, 'step': 1500}



{'eval_loss': 3.4495080650829877, 'epoch': 2.767527675276753, 'step': 1500}




TrainOutput(global_step=1626, training_loss=3.4998997373627856)

In [18]:
trainer.evaluate()


{'eval_loss': 3.446699173724065, 'epoch': 3.0, 'step': 1626}


{'eval_loss': 3.446699173724065, 'epoch': 3.0}

In [19]:
tokenizer.save_pretrained('/kaggle/working/model/')

('/kaggle/working/model/vocab.json',
 '/kaggle/working/model/merges.txt',
 '/kaggle/working/model/special_tokens_map.json',
 '/kaggle/working/model/added_tokens.json')

In [20]:
model.save_pretrained('/kaggle/working/model/')